# Import & Load Election participation data

### Config

In [1]:
import sys,os
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [2]:
remote_path = 'http://pacha.datawheel.us/'
local_path = '../data/'

### Imports

In [3]:
import postgres #from local file postgres.py
from commons import inline_table_xml, inline_dimension_xml, download_file, download_zip_file, extract_zip_file #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load file

In [4]:
download_zip_file(remote_path,
                  local_path,
                  "participacion-presidenciales-2017.zip")

extract_zip_file(local_path, "temp.zip")

Downloading... http://pacha.datawheel.us/participacion-presidenciales-2017.zip
Unzipping... ../data/temp.zip


True

In [5]:
df = pd.read_csv(os.path.join(local_path, "participacion_total.csv"),low_memory=True)
df = df.astype({'comuna_datachile_id':'int', 'year':'int','election_id':'int','total_electores':'int','total_votacion':'int'})
list(df)

['year',
 'election_id',
 'comuna_datachile_id',
 'total_electores',
 'total_votacion']

### Ingest

In [7]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'politics', 'fact_election_participation')

DROP TABLE IF EXISTS politics.fact_election_participation;
CREATE TABLE "politics"."fact_election_participation" (
"year" INTEGER,
  "election_id" INTEGER,
  "comuna_datachile_id" INTEGER,
  "total_electores" INTEGER,
  "total_votacion" INTEGER
)
COPY "politics"."fact_election_participation" ("year","election_id","comuna_datachile_id","total_electores","total_votacion") FROM STDIN WITH CSV HEADER DELIMITER ',';


### Indexes & FK

In [9]:
engine.execute("""
CREATE INDEX fact_election_participation_comuna_datachile_id 
ON politics.fact_election_participation (comuna_datachile_id)
""")

engine.execute("""
CREATE INDEX fact_election_participation_election_id 
ON politics.fact_election_participation (election_id)
""")

engine.execute("""
CREATE INDEX fact_election_participation_year 
ON politics.fact_election_participation (year)
""")



### Related dim

* Comuna
* Election Type